In [1]:
# 之前的分析一直是词袋分析，没有把句子的先后顺序反应出来
# 然后看了一些博客，paper，sentence2vec虽然也有，但不过是用word2vec的方法在句子层面上做了一遍
# 也有一些基于word2vec+rnn的句子相似度模型，其实这个挺不错，但是我目前的tensorflow基础好像还做不到
# 最终决定先做一个word2vec，然后用词向量的简单平均求和看看，顺便也复习一下tensorflow里怎么实现word2vec

In [3]:
import pandas as pd
import numpy as np
filepath = 'C:/Users\Administrator\Desktop/'

In [4]:
train_data = pd.read_csv(filepath+'train.csv')
test_data = pd.read_csv(filepath+'test.csv')

In [18]:
data_combine = pd.concat([train_data,test_data],axis=0)

In [19]:
from nltk import WordNetLemmatizer
from nltk import PorterStemmer
from nltk import word_tokenize

In [20]:
data_combine['word_split'] = data_combine.text.apply(lambda x:word_tokenize(str(x)))

In [24]:
wordlist = []
for i in data_combine.word_split:
    for x in i:
        if x not in wordlist:
            wordlist.append(x)

In [26]:
vocab_size = len(wordlist)

In [28]:
# 建立两个字典
word2int = {}
int2word = {}
for i,i_index in enumerate(wordlist):
    word2int[i]=i_index
    int2word[i_index] = i

In [36]:
import tensorflow as tf
import math

In [261]:
embed_size = 100
batch_size = None
win = 3
epoch = 60000
n=0
pp=[[],[]]

In [262]:
graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder(shape=[batch_size],dtype=tf.int32)
    labels = tf.placeholder(shape=[batch_size,1],dtype=tf.int32)
    embed_dict = tf.Variable(
        tf.random_uniform(shape=[vocab_size,embed_size],minval=-1.0,maxval=1.0,name='vector')
    )
    nce_weight = tf.Variable(
        tf.truncated_normal(shape=[vocab_size,embed_size],stddev=1.0/math.sqrt(embed_size))
    )
    nce_bias = tf.Variable(tf.zeros([vocab_size]))
        
    embed = tf.nn.embedding_lookup(embed_dict,inputs)
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weight,biases=nce_bias,labels=labels,num_sampled=100,num_classes=vocab_size,inputs=embed)
    )
    train_op = tf.train.GradientDescentOptimizer(learning_rate=0.04).minimize(loss)     
    init = tf.global_variables_initializer()

In [263]:
def train(input_sentence,n=n):
        batch_input = []
        batch_label = []
        sent_len = len(input_sentence)
        for i in range(sent_len):
            start = max(0,i-win)
            end = min(sent_len,i+win)
            for index in range(start,end):
                if index==i:
                    continue
                else:
                    input_id = int2word[input_sentence[i]]
                    label_id = int2word[input_sentence[index]]
                    batch_input.append(input_id)
                    batch_label.append(label_id)
        if len(batch_input)==0:
            return
        batch_input = np.array(batch_input,dtype=np.int32)
        batch_label = np.array(batch_label,dtype=np.int32)
        batch_label = np.reshape(batch_label,[len(batch_label),1])
        feed_dic = {inputs:batch_input,labels:batch_label}
        _,loss_val = sess.run([train_op,loss],feed_dict=feed_dic)
        n+=1
        if n%100==0:
            pp[0].append(n)
            pp[1].append(loss_val)
        return n

In [264]:
sess = tf.Session(graph=graph)
sess.run(init)
with sess.as_default():
    for j in range(2):
        for i in data_combine.word_split:
            if n<epoch:
                n = train(i,n)
    final_embeddings = embed_dict.eval()

In [268]:
final_embed = pd.DataFrame(final_embeddings,columns=['embed_'+str(i) for i in range(embed_size)])

In [270]:
def embed(x,num):
    sum = 0
    l = 0
    for i in x:
        index = int2word[i]
        sum+=final_embed['embed_'+str(num)][index]
        l+=1
    return sum/l
for i in range(embed_size):
    data_combine['embed_mean'+str(i)] = data_combine.word_split.apply(lambda x:embed(x,i))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [271]:
data_combine

,author,id,text,word_split,embed_mean0,embed_mean1,embed_mean2,embed_mean3,embed_mean4,embed_mean5,...,embed_mean90,embed_mean91,embed_mean92,embed_mean93,embed_mean94,embed_mean95,embed_mean96,embed_mean97,embed_mean98,embed_mean99
0,EAP,id26305,"This process, however, afforded me no means of...","[This, process, ,, however, ,, afforded, me, n...",0.070504,-0.720657,-0.124517,0.048419,-1.257137,0.927735,...,-0.362372,0.848026,-0.758443,-0.499002,0.820771,-0.141672,-0.876783,-0.740274,0.998730,0.382432
1,HPL,id17569,It never once occurred to me that the fumbling...,"[It, never, once, occurred, to, me, that, the,...",-0.080710,-0.824791,0.175443,-0.283660,-1.079248,0.620678,...,-0.208425,0.869166,-0.766891,-0.508036,0.963736,-0.329598,-0.966869,-0.363421,0.884524,0.065513
2,EAP,id11008,"In his left hand was a gold snuff box, from wh...","[In, his, left, hand, was, a, gold, snuff, box...",-0.042085,-0.703357,0.182896,0.239447,-1.169754,0.611267,...,-0.255667,0.732252,-0.691535,-0.579491,0.945585,-0.082390,-1.007435,-0.571315,0.863602,0.370459
3,MWS,id27763,How lovely is spring As we looked from Windsor...,"[How, lovely, is, spring, As, we, looked, from...",0.103623,-0.561895,0.165274,-0.028690,-0.990677,0.491261,...,-0.218475,0.576568,-0.634810,-0.502013,0.724672,0.051548,-0.915522,-0.419592,0.949740,0.217544
4,HPL,id12958,"Finding nothing else, not even gold, the Super...","[Finding, nothing, else, ,, not, even, gold, ,...",-0.014432,-0.699222,0.198795,-0.024548,-1.172243,0.769937,...,-0.238642,0.619914,-0.968461,-0.299549,0.762025,-0.137790,-0.839535,-0.523210,0.805924,0.501715
5,MWS,id22965,"A youth passed in solitude, my best years spen...","[A, youth, passed, in, solitude, ,, my, best, ...",0.012616,-0.683830,0.004066,0.020275,-1.112295,0.737762,...,-0.238617,0.669975,-0.696223,-0.415651,0.916130,-0.186277,-0.873063,-0.555418,0.989591,0.361738
6,EAP,id09674,"The astronomer, perhaps, at this point, took r...","[The, astronomer, ,, perhaps, ,, at, this, poi...",0.007611,-0.662410,-0.061389,0.279349,-1.159941,0.820284,...,-0.308443,0.651980,-0.760818,-0.483205,0.757944,-0.010748,-1.063297,-0.606575,0.863577,0.163695
7,EAP,id13515,The surcingle hung in ribands from my body.,"[The, surcingle, hung, in, ribands, from, my, ...",0.347278,-0.664871,-0.129348,0.030635,-1.056840,0.623063,...,-0.510477,0.580777,-0.628862,-0.497022,0.991200,0.173140,-0.713423,-0.512007,0.683255,0.196236
8,EAP,id19322,I knew that you could not say to yourself 'ste...,"[I, knew, that, you, could, not, say, to, your...",0.022410,-0.669268,-0.003385,0.100006,-1.198162,0.771807,...,-0.375631,0.806294,-0.782904,-0.485452,0.869399,-0.011221,-0.866119,-0.574630,1.013981,0.350330
9,MWS,id00912,I confess that neither the structure of langua...,"[I, confess, that, neither, the, structure, of...",-0.034146,-0.639516,-0.025414,0.169895,-0.865644,0.742903,...,-0.334297,0.581079,-0.542186,-0.496106,0.770538,-0.123067,-0.849957,-0.595343,0.685500,0.497221


In [272]:
writer = pd.ExcelWriter('D:/大学/大三上/数据挖掘/作业/数据embed.xlsx')
data_combine.to_excel(writer,sheet_name='sheet1')
writer.save()